# Inference and evaluation on the test set, for each model

# Pip installments

In [ ]:
! pip install datasets

! pip install evaluate

! pip install --upgrade tensorflow

! pip install rouge_score

!pip install bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.2 MB/s eta 0:00:00
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.15.0
    Uninstalling tensorflow-2.15.0:
      Successfully uninstalled tensorflow-2.15.0
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=7d7c103783b593fc3feb9c83ae38b0f6a7ed0b0768dbc9a29b2b2f52411510ef
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
! pip install nltk

# Libraries

In [ ]:
import numpy as np

from matplotlib import pyplot as plt

import pandas as pd

In [ ]:
#nltk basics
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import names
import nltk

# Download the necessary resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('names')

# Import the punctuation module
from string import punctuation

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Unzipping corpora/names.zip.


# Evaluation metrics

In [ ]:
from evaluate import load
# Load the ROUGE metric
import evaluate
rouge = evaluate.load('rouge')

In [ ]:
bleu = evaluate.load("bleu")

In [ ]:
meteor = evaluate.load('meteor')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
bertscore = load("bertscore")

# Dataset (code_x_glue_ct_code_to_text)

- LINK: https://huggingface.co/datasets/code_x_glue_ct_code_to_text/viewer/ruby

In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("code_x_glue_ct_code_to_text", "ruby")
print(dataset)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/24927 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1400 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1261 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url'],
        num_rows: 24927
    })
    validation: Dataset({
        features: ['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url'],
        num_rows: 1400
    })
    test: Dataset({
        features: ['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url'],
        num_rows: 1261
    })
})


# Inference

- The inference for each model is performed on the test set and then evaluated

- **TPU** runtime


# Inference code-t5-multi-sum

LINK: https://huggingface.co/Salesforce/codet5-base-multi-sum



Loading directly from HuggingFace Hub

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Salesforce/codet5-base-multi-sum")
model = AutoModelForSeq2SeqLM.from_pretrained("Salesforce/codet5-base-multi-sum")

tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/902 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

## Batch processing

-  `batch_inference()` function

In [ ]:
def batch_inference(input_dataset, batch_size=20):

  #Extract the methods and the ground_truths(labels)
  methods = [input_dataset["test"]["code"][id] for id, _ in enumerate(input_dataset["test"]["code"])]
  labels = [input_dataset["test"]["docstring"][id] for id, _ in enumerate(input_dataset["test"]["docstring"])]

  #Initializing and defining variables
  max_input_length = 256
  final_output = []

  #Start the batch processing + inference + decoding
  for i in range(0,len(methods), batch_size):

    #Define the batch
    batch = methods[i:i+batch_size]

    #Tokenize the batch
    inputs = tokenizer(batch, max_length=max_input_length, padding="max_length", truncation=True, return_tensors="pt")

    #Inference the batch
    outputs = model.generate(**inputs, max_length=60, min_length=30)

    #Decoding the batch
    summaries = [tokenizer.decode(summary, skip_special_tokens=True) for summary in outputs]
    final_output.append(summaries)

  #merge the sublists
  summy = sum(final_output, [])

  return (summy, labels[0:len(summy)])


Perform the inference

In [ ]:
summary_results, ground_truth = batch_inference(dataset, batch_size=20)

Save the results in a dataframe

In [ ]:
output_dir = "path..."

df_result = pd.DataFrame({"summary_results": summary_results, "ground_truth": ground_truth})

df_result.to_csv( output_dir + "summaries_codet5_multi_sum.csv", index=False)

In [ ]:
df_result.head()

,summary_results,ground_truth
0,Prints the summary of a single branch. This is...,Outputs the deferred summary information saved...
1,This method is called to find bad files from t...,Inspect the file referenced in the kubectl std...
2,Checks if the EJSON keys are not pruned. If so...,make sure to never prune the ejson-keys secret
3,Returns the index of the node with the given n...,Determine the best compressor for the current ...
4,Create a. bff file in staging_dir where the fi...,Create the bff file using +mkinstallp+.\n\n Wa...


## Evaluation

For the evaluation of the summaries computed, the following metrics were used. To assess the quality of the summary the metrics perform different kind of comparisons between the summary and the ground truth(docstring); aiming to assess some kind of similarity.

- ROUGE `rouge_summary(df)`
  - stemmer is used

- BLEU, `bleu_summary(df)`
  - word_tokenizer from nltk
  - geometric mean of n-gram precisions

- METEOR, `meteor_summary(df)`

- DistilBert_score, `bert_summary(df)`
  - It leverages BERT embeddings to measure the similarity between the generated summary and the reference summaries.
  - BertScore has shown effectiveness in capturing semantic similarity.

In [ ]:
def rouge_summary(df):
  summary_results = df.summary_results.tolist()
  ground_truth = df.ground_truth.tolist()

  #rouge computation (3min circa)
  values = rouge.compute(predictions = summary_results, references = ground_truth, use_aggregator= False, use_stemmer=True)

  metrics = ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']
  for elem in metrics:
    print(f"{elem}  mean:{np.round(np.mean(values[elem]), 2)}  std:{np.round(np.std(values[elem]), 2)} ")


In [ ]:
rouge_summary(df_result)

rouge1  mean:0.23  std:0.11 
rouge2  mean:0.06  std:0.07 
rougeL  mean:0.19  std:0.09 
rougeLsum  mean:0.21  std:0.11 


In [ ]:
def bleu_summary(df):
  summary_results = df.summary_results.tolist()
  ground_truth = df.ground_truth.tolist()

  values = [bleu.compute(predictions=[summary_results[id]], references=[[ground_truth[id]]], tokenizer=word_tokenize, smooth=True)["bleu"] for id in range(0, df.shape[0])]

  precis_0 = [bleu.compute(predictions=[summary_results[id]], references=[[ground_truth[id]]], tokenizer=word_tokenize, smooth=True)["precisions"][0] for id in range(0, df.shape[0])]
  precis_1 = [bleu.compute(predictions=[summary_results[id]], references=[[ground_truth[id]]], tokenizer=word_tokenize, smooth=True)["precisions"][1] for id in range(0, df.shape[0])]
  precis_2 = [bleu.compute(predictions=[summary_results[id]], references=[[ground_truth[id]]], tokenizer=word_tokenize, smooth=True)["precisions"][2] for id in range(0, df.shape[0])]
  precis_3 = [bleu.compute(predictions=[summary_results[id]], references=[[ground_truth[id]]], tokenizer=word_tokenize, smooth=True)["precisions"][3] for id in range(0, df.shape[0])]

  print(f"  Bleu: mean:{np.round(np.mean(values),4)}, std:{np.round(np.std(values),4)}\nunigram_mean:{np.round(np.mean(precis_0),4)}; unigram_std:{np.round(np.std(precis_0),4)}\nbigram_mean:{np.round(np.mean(precis_1),4)}; bigram_std:{np.round(np.std(precis_1),4)}\ntrigram_mean:{np.round(np.mean(precis_2),4)}; trigram_std:{np.round(np.std(precis_2),4)}\nfourgram_mean:{np.round(np.mean(precis_3),4)}; fourgram_std:{np.round(np.std(precis_3),4)}\n")


In [ ]:
bleu_summary(df_result)

  Blue: mean:0.0508, std:0.0315
unigram_mean:0.2206; unigram_std:0.1485
bigram_mean:0.0733; bigram_std:0.0557
trigram_mean:0.0436; trigram_std:0.0294
fourgram_mean:0.0366; fourgram_std:0.0184



In [ ]:
def meteor_summary(df):
  summary_results = df.summary_results.tolist()
  ground_truth = df.ground_truth.tolist()

  values = [meteor.compute(predictions=[summary_results[id]], references=[[ground_truth[id]]])["meteor"] for id in range(0, df.shape[0])]

  print(f" Meteor: mean:{np.round(np.mean(values),4)}, std:{np.round(np.std(values),4)}")


In [ ]:
meteor_summary(df_result)

 Meteor: mean:0.1776, std:0.1104'


In [ ]:
def bertscore_summary(df):
  summary_results = df.summary_results.tolist()
  ground_truth = df.ground_truth.tolist()

  values = bertscore.compute(predictions=summary_results, references=ground_truth, lang="en", model_type="distilbert-base-uncased")

  metrics = ['precision', 'recall', 'f1']
  for elem in metrics:
    print(f"{elem}  mean:{np.round(np.mean(values[elem]), 2)}  std:{np.round(np.std(values[elem]), 2)} ")


In [ ]:
bertscore_summary(df_result)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

precision  mean:0.77  std:0.05 
recall  mean:0.77  std:0.06 
f1  mean:0.77  std:0.04 


# Inference code-t5-small-RUBY fine-tuned

Load the tokenizer, and the model

- The fine tuned model has to be retrieved from the directory used for saving it, after the fine tuning

In [ ]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

checkpoint_directory = "path..."

tokenizer = AutoTokenizer.from_pretrained("Salesforce/codet5-small")
model = T5ForConditionalGeneration.from_pretrained(checkpoint_directory)

Perform the inference

In [ ]:
summary_results, ground_truth = batch_inference(dataset, batch_size=20)

Save the results in a dataframe

In [ ]:
output_dir = "path..."

df_result = pd.DataFrame({"summary_results": summary_results, "ground_truth": ground_truth})

df_result.to_csv(output_dir+"summaries_t5_small_RubyTuned.csv", index=False)

Evaluation

In [ ]:
rouge_summary(df_result)

rouge1  mean:0.28  std:0.15 
rouge2  mean:0.09  std:0.1 
rougeL  mean:0.24  std:0.13 
rougeLsum  mean:0.27  std:0.15 


In [ ]:
bleu_summary(df_result)

  Bleu: mean:0.0803, std:0.0773
unigram_mean:0.2823; unigram_std:0.2196
bigram_mean:0.1342; bigram_std:0.1418
trigram_mean:0.086; trigram_std:0.0993
fourgram_mean:0.0637; fourgram_std:0.072



In [ ]:
meteor_summary(df_result)

 Meteor: mean:0.2268, std:0.1542


In [ ]:
bertscore_summary(df_result)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

precision  mean:0.77  std:0.08 
recall  mean:0.8  std:0.06 
f1  mean:0.78  std:0.06 


# Inference code T5-multi-sum fine tuned

Load the tokenizer and the model

- The fine tuned model has to be retrieved from the directory used for saving it, after the fine tuning

In [ ]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

checkpoint_directory='path...'

tokenizer = AutoTokenizer.from_pretrained("Salesforce/codet5-base-multi-sum")

model = T5ForConditionalGeneration.from_pretrained(checkpoint_directory)

Perform the inference

In [ ]:
summary_results, ground_truth = batch_inference(dataset, batch_size=20)

Save the results in a dataframe

In [ ]:
output_dir = "path..."

df_result = pd.DataFrame({"summary_results": summary_results, "ground_truth": ground_truth})

df_result.to_csv(output_dir+"summaries_t5_multisum_rubyTuned.csv", index=False)

Evaluation

In [ ]:
rouge_summary(df_result)

rouge1  mean:0.29  std:0.15 
rouge2  mean:0.1  std:0.11 
rougeL  mean:0.25  std:0.13 
rougeLsum  mean:0.28  std:0.15 


In [ ]:
bleu_summary(df_result)

  Bleu: mean:0.0834, std:0.081
unigram_mean:0.2886; unigram_std:0.218
bigram_mean:0.1378; bigram_std:0.1421
trigram_mean:0.089; trigram_std:0.1012
fourgram_mean:0.0654; fourgram_std:0.0753



In [ ]:
meteor_summary(df_result)

 Meteor: mean:0.2348, std:0.1573


In [ ]:
bertscore_summary(df_result)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

precision  mean:0.78  std:0.08 
recall  mean:0.8  std:0.06 
f1  mean:0.79  std:0.06 
